# Reacting to Button Presses with Asyncio

We already introduced one, simple way of reacting to button presses in the [main notebook](../2_board_io.ipynb). This will be enough for many applications, but it does have some restrictions:

  1. We can't do anything else in Python while we wait for a button to change value
  2. We can only wait for one button at a time
  
If we want to listen to different buttons simultaneously or we have some background task that must be run (e.g. updating an LED pattern), we will need to use a more complex approach. We'll introduce an approach using concurrent/asynchronous programming and we won't lie... this can be **hard**.

There are parts of this that we don't expect you to fully understand, but we will highlight the place to put in your own custom code so you can use this for your own projects. If you're particularly interested in the hairy details, please ask one of the training helpers!

First let's define what we want to happen when a button is pressed and released. We'll loop forever (`while True:` does this) and write some actions for button press and release. For now, let's print a message and turn the corresponding LED on, but you could add in your own actions here too.

In [2]:
from pynq.overlays.base import BaseOverlay

board = BaseOverlay("base.bit")

In [3]:
async def detect_button_press(number):
    
    button = board.buttons[number]
    led    = board.leds[number]
    
    while True:
        
        # Wait here until the button is pressed
        await button.wait_for_value_async(1)
        
        # OK, button is pressed — add any extra code here!
        print('Button '+ str(number) + ' pressed  😄')
        led.on()
        
        # Wait here until the button is released
        await button.wait_for_value_async(0)
        
        # OK, button is released — add any extra code here!
        print('Button '+ str(number) + ' released 😔')
        led.off()

Note that there are 2 new keywords in this cell: `async` and `await`.

  * `async` tells Python that it is OK to run the following code *asynchronously*. This means that the execution can be paused, something else can be run for a while, and then our code can be happily resumed as if nothing happened.
  
  * `await` tells Python that we'll likely need to wait a while for this line to give us a value. During this wait, it is OK for other `async` code to be executed.

Now that we've defined what should happen when a button is pressed, let's run one of these "tasks" for each button. This will loop forever, so press the stop button (⏹️) in the toolbar when you want to move on.

In [4]:
import asyncio

# Let's make one "task" for each push button
for i in range(4):
    asyncio.ensure_future(detect_button_press(i))
    
# Start running all of these tasks!
try:
    asyncio.get_event_loop().run_forever()
    
# ...Until you press the stop button
except KeyboardInterrupt:
    print ('Stopping event loop')
    
# ...Then politely stop all of the tasks
finally:
    for task in asyncio.Task.all_tasks():
        task.cancel()

Button 3 pressed  😄
Button 1 pressed  😄
Button 3 released 😔
Button 1 released 😔
Stopping event loop


Note that we can now react to any button at all — not just `BTN0`.

If you want to add in a background task, we can just add it with `asyncio.ensure_future(...)`. Just as an example, let's do the same as above but with an extra task that prints a message once every second. Note that we need to use a special version of `sleep()` when using `asyncio`.

In [6]:
# Define our background task
async def background_task():
    while True:
        await asyncio.sleep(1)
        print('⏰ Hello from the background task! ⏰')

# Add our background task to the system
asyncio.ensure_future(background_task())

# All code below is the same as previous cell...

for i in range(4):
    asyncio.ensure_future(detect_button_press(i))
    
try:
    asyncio.get_event_loop().run_forever()
except KeyboardInterrupt:
    print ('Stopping event loop')
finally:
    for task in asyncio.Task.all_tasks():
        task.cancel()

⏰ Hello from the background task! ⏰
⏰ Hello from the background task! ⏰
⏰ Hello from the background task! ⏰
⏰ Hello from the background task! ⏰
⏰ Hello from the background task! ⏰
⏰ Hello from the background task! ⏰
Button 1 pressed  😄
⏰ Hello from the background task! ⏰
⏰ Hello from the background task! ⏰
Button 1 released 😔
⏰ Hello from the background task! ⏰
Button 0 pressed  😄
⏰ Hello from the background task! ⏰
⏰ Hello from the background task! ⏰
Button 0 released 😔
⏰ Hello from the background task! ⏰
⏰ Hello from the background task! ⏰
Stopping event loop


This code might look a little intimidating but you can reuse it for your own project just by changing a line or two where we say "`add any extra code here!`".

-------------------------------------------------------------

[Click here](../2_board_io.ipynb) to go back to the main notebook.